This tutorial describes how to evaluate rules that are applicable to two consecutive periods (year and quarter).

In [1]:
from arelle import ModelManager, Cntlr, ModelFormulaObject, ModelXbrl, ViewFileFormulae, XbrlConst, ViewFileRenderedGrid
from arelle import RenderingEvaluator 

In [2]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import join, isfile
import pickle
import re
from src import Evaluator
import logging
import data_patterns
import datetime

In [3]:
DECIMALS = 0
RULES_PATH = join('..', 'solvency2-rules')
INSTANCES_DATA_PATH = join('..','data','instances')
DATAPOINTS_PATH = join('..', 'data', 'datapoints')

### Import rules

We start with importing the (t-1)-t rules that are applicable to two consecutive periods. We import a set of rules used to evaluate year data and a set of rules for quarter data.

#### S2_betweenperiods_ARS

In [4]:
dfr_ARS = pd.read_excel(join(RULES_PATH,'S2_betweenperiods_ARS.xlsx'), engine='openpyxl')

#Capitalize row-column references:
column_replace = set([column for sublist in [row for row in dfr_ARS['pandas ex'].str.findall(r'c\d\d\d\d')] for column in sublist])
for ref in column_replace:
    dfr_ARS.replace(to_replace=ref, value=ref.capitalize(), inplace=True, regex=True)
column_replace = set([column for sublist in [row for row in dfr_ARS['pandas ex'].str.findall(r'r\d\d\d\d')] for column in sublist])
for ref in column_replace:
    dfr_ARS.replace(to_replace=ref, value=ref.capitalize(), inplace=True, regex=True)
dfr_ARS

,Unnamed: 0,pattern_id,cluster,pattern_def,support,exceptions,confidence,pattern status,encodings,pandas co,pandas ex,xbrl co,xbrl ex,Error message
0,0,"S.01.02.01.01,R0991,C0010 consistently reported",Schade,"(({""S.01.02.01.01,R0991,C0010 (t)""} != 0) & ({...",285,0,1.0000,statistical validation rule,{},"df[((((((df[""S.01.02.01.01,R0991,C0010 (t)""]!=...","df[~((((((df[""S.01.02.01.01,R0991,C0010 (t)""]!...",NaN,NaN,NaN
1,1,"S.01.02.01.01,R0991,C0010 consistently reported",Herverzekeraar,"(({""S.01.02.01.01,R0991,C0010 (t)""} != 0) & ({...",25,0,1.0000,statistical validation rule,{},"df[((((((df[""S.01.02.01.01,R0991,C0010 (t)""]!=...","df[~((((((df[""S.01.02.01.01,R0991,C0010 (t)""]!...",NaN,NaN,NaN
2,2,"S.01.02.01.01,R0991,C0010 consistently reported",Leven,"(({""S.01.02.01.01,R0991,C0010 (t)""} != 0) & ({...",80,0,1.0000,statistical validation rule,{},"df[((((((df[""S.01.02.01.01,R0991,C0010 (t)""]!=...","df[~((((((df[""S.01.02.01.01,R0991,C0010 (t)""]!...",NaN,NaN,NaN
3,3,"S.01.02.01.01,R0992,C0010 consistently reported",Schade,"(({""S.01.02.01.01,R0992,C0010 (t)""} != 0) & ({...",285,0,1.0000,statistical validation rule,{},"df[((((((df[""S.01.02.01.01,R0992,C0010 (t)""]!=...","df[~((((((df[""S.01.02.01.01,R0992,C0010 (t)""]!...",NaN,NaN,NaN
4,4,"S.01.02.01.01,R0992,C0010 consistently reported",Herverzekeraar,"(({""S.01.02.01.01,R0992,C0010 (t)""} != 0) & ({...",25,0,1.0000,statistical validation rule,{},"df[((((((df[""S.01.02.01.01,R0992,C0010 (t)""]!=...","df[~((((((df[""S.01.02.01.01,R0992,C0010 (t)""]!...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14150,14150,"S.29.03.01.07,R0360,C0130 consistently reported",Herverzekeraar,"(({""S.29.03.01.07,R0360,C0130 (t)""} != 0) & ({...",17,0,1.0000,statistical validation rule,{},"df[((((((df[""S.29.03.01.07,R0360,C0130 (t)""]!=...","df[~((((((df[""S.29.03.01.07,R0360,C0130 (t)""]!...",NaN,NaN,NaN
14151,14151,"S.29.03.01.07,R0360,C0130 consistently reported",Leven,"(({""S.29.03.01.07,R0360,C0130 (t)""} != 0) & ({...",54,0,1.0000,statistical validation rule,{},"df[((((((df[""S.29.03.01.07,R0360,C0130 (t)""]!=...","df[~((((((df[""S.29.03.01.07,R0360,C0130 (t)""]!...",NaN,NaN,NaN
14152,14152,"S.29.03.01.07,R0370,C0130 consistently reported",Schade,"(({""S.29.03.01.07,R0370,C0130 (t)""} != 0) & ({...",198,3,0.9851,statistical validation rule,{},"df[((((((df[""S.29.03.01.07,R0370,C0130 (t)""]!=...","df[~((((((df[""S.29.03.01.07,R0370,C0130 (t)""]!...",NaN,NaN,NaN
14153,14153,"S.29.03.01.07,R0370,C0130 consistently reported",Herverzekeraar,"(({""S.29.03.01.07,R0370,C0130 (t)""} != 0) & ({...",17,0,1.0000,statistical validation rule,{},"df[((((((df[""S.29.03.01.07,R0370,C0130 (t)""]!=...","df[~((((((df[""S.29.03.01.07,R0370,C0130 (t)""]!...",NaN,NaN,NaN


#### S2_betweenperiods_QRS

In [5]:
dfr_QRS = pd.read_excel(join(RULES_PATH,'S2_betweenperiods_QRS.xlsx'), engine='openpyxl')

#Capitalize row-column references:
column_replace = set([column for sublist in [row for row in dfr_QRS['pandas ex'].str.findall(r'c\d\d\d\d')] for column in sublist])
for ref in column_replace:
    dfr_QRS.replace(to_replace=ref, value=ref.capitalize(), inplace=True, regex=True)
column_replace = set([column for sublist in [row for row in dfr_QRS['pandas ex'].str.findall(r'r\d\d\d\d')] for column in sublist])
for ref in column_replace:
    dfr_QRS.replace(to_replace=ref, value=ref.capitalize(), inplace=True, regex=True)
dfr_QRS

,Unnamed: 0,pattern_id,cluster,pattern_def,support,exceptions,confidence,pattern status,encodings,pandas co,pandas ex,xbrl co,xbrl ex,Error message
0,0,"S.01.02.01.01,R0990,C0010 consistently reported",Schade,"(({""S.01.02.01.01,R0990,C0010 (t)""} != 0) & ({...",1771,0,1.0,statistical validation rule,{},"df[((((((df[""S.01.02.01.01,R0990,C0010 (t)""]!=...","df[~((((((df[""S.01.02.01.01,R0990,C0010 (t)""]!...",NaN,NaN,NaN
1,1,"S.01.02.01.01,R0990,C0010 consistently reported",Leven,"(({""S.01.02.01.01,R0990,C0010 (t)""} != 0) & ({...",508,0,1.0,statistical validation rule,{},"df[((((((df[""S.01.02.01.01,R0990,C0010 (t)""]!=...","df[~((((((df[""S.01.02.01.01,R0990,C0010 (t)""]!...",NaN,NaN,NaN
2,2,"S.01.02.01.01,R0990,C0010 consistently reported",Herverzekeraar,"(({""S.01.02.01.01,R0990,C0010 (t)""} != 0) & ({...",155,0,1.0,statistical validation rule,{},"df[((((((df[""S.01.02.01.01,R0990,C0010 (t)""]!=...","df[~((((((df[""S.01.02.01.01,R0990,C0010 (t)""]!...",NaN,NaN,NaN
3,3,"S.01.02.01.01,R0991,C0010 consistently reported",Schade,"(({""S.01.02.01.01,R0991,C0010 (t)""} != 0) & ({...",1771,0,1.0,statistical validation rule,{},"df[((((((df[""S.01.02.01.01,R0991,C0010 (t)""]!=...","df[~((((((df[""S.01.02.01.01,R0991,C0010 (t)""]!...",NaN,NaN,NaN
4,4,"S.01.02.01.01,R0991,C0010 consistently reported",Leven,"(({""S.01.02.01.01,R0991,C0010 (t)""} != 0) & ({...",508,0,1.0,statistical validation rule,{},"df[((((((df[""S.01.02.01.01,R0991,C0010 (t)""]!=...","df[~((((((df[""S.01.02.01.01,R0991,C0010 (t)""]!...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2860,2860,"S.28.02.01.06,R0520,C0150 consistently reported",Herverzekeraar,"(({""S.28.02.01.06,R0520,C0150 (t)""} != 0) & ({...",6,0,1.0,statistical validation rule,{},"df[((((((df[""S.28.02.01.06,R0520,C0150 (t)""]!=...","df[~((((((df[""S.28.02.01.06,R0520,C0150 (t)""]!...",NaN,NaN,NaN
2861,2861,"S.28.02.01.06,R0530,C0150 consistently reported",Herverzekeraar,"(({""S.28.02.01.06,R0530,C0150 (t)""} != 0) & ({...",6,0,1.0,statistical validation rule,{},"df[((((((df[""S.28.02.01.06,R0530,C0150 (t)""]!=...","df[~((((((df[""S.28.02.01.06,R0530,C0150 (t)""]!...",NaN,NaN,NaN
2862,2862,"S.28.02.01.06,R0540,C0150 consistently reported",Herverzekeraar,"(({""S.28.02.01.06,R0540,C0150 (t)""} != 0) & ({...",6,0,1.0,statistical validation rule,{},"df[((((((df[""S.28.02.01.06,R0540,C0150 (t)""]!=...","df[~((((((df[""S.28.02.01.06,R0540,C0150 (t)""]!...",NaN,NaN,NaN
2863,2863,"S.28.02.01.06,R0550,C0150 consistently reported",Herverzekeraar,"(({""S.28.02.01.06,R0550,C0150 (t)""} != 0) & ({...",6,0,1.0,statistical validation rule,{},"df[((((((df[""S.28.02.01.06,R0550,C0150 (t)""]!=...","df[~((((((df[""S.28.02.01.06,R0550,C0150 (t)""]!...",NaN,NaN,NaN


### Import templates

Next we import the reporting data. We import the data of two consecutive periods. In the tutorial 'Convert XBRL-instances to CSV, HTML and pickles' the XBRL-instances are converted to pickle files per template. The pickle files are written to the data/instances folder. The rules are applicable to all tables with closed axis. We import these pickle files. When comparing two periods it can be the case that two different taxonomies are applicable. The right taxonomy has to be selected in the tutorial 'Convert XBRL-instances to CSV, HTML and pickles' to convert the XBRL-instance properly. 

The list _instances_ARS_ contains the names of the folders with the converted XBRL-instance for yearly data. The list _instances_QRS_ contains the names of the folders with the converted XBRL-instance for two consecutive quarters. Finally, we also have to define the category of the insurer. The rules are set-up for each type of insurer separately.

In [6]:
instances_ARS = []
instances_QRS = []
categorie = '' #which type of insurer the instance belongs to (Schade, Herverzekeraar, Leven)

#### S2_betweenperiods_ARS

In [7]:
# df_datapoints = pd.read_csv(join(DATAPOINTS_PATH, 'ARS.csv'), sep=";").fillna("")  # load file to dataframe
# dft = pd.DataFrame()
# for instance in instances_ARS:
#     df_closed_axis = pd.DataFrame()
#     tables_closed_axis = []  # for listing all input tables with closed axis
#     tables_complete_set = df_datapoints.tabelcode.sort_values().unique().tolist()  # list of all ARS tables
#     tables = [table for table in tables_complete_set 
#         if isfile(join(INSTANCES_DATA_PATH,instance,table + '.pickle'))]  # ARS tables found in the specified instance path
#     for table in [table for table in tables if table not in ['S.14.01.01.04','S.30.03.01.01']]:  #tables:
#         if isfile(join(INSTANCES_DATA_PATH,instance, table + '.pickle')):
#             df = pd.read_pickle(join(INSTANCES_DATA_PATH,instance, table + '.pickle'))  # read dataframe
#         else:
#             continue   
#         if df.index.nlevels > 2:  # if more than 2 indexes (entity, period), then the table has an open axis
#             continue
#         else:  # closed axis
#             tables_closed_axis.append(table)  # add to relevant list
#             # Add table to dataframe with all data from closed axis tables
#             if len(df_closed_axis) == 0:  # no data yet --> copy dataframe
#                 df_closed_axis = df.copy()
#             else:  # join to existing dataframe
#                 df_closed_axis = df_closed_axis.join(df)
#     if len(dft) == 0:  # no data yet 
#         dft = df_closed_axis
#     else:  # join to existing dataframe
#         dft=dft.append(df_closed_axis)
# dft=dft.reset_index()
# dft['categorie']=categorie
# numerical_columns = ['entity','period','categorie'] + [dft.columns[c] for c in range(len(dft.columns))
#                         if ((dft.dtypes[c] == 'float64') or (dft.dtypes[c] == 'int64'))] #select only numerical columns
# df_ARS = dft[numerical_columns]
# df_ARS['period']=df_ARS['period'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d')) #convert to datetime
# df_ARS.fillna(0,inplace=True)

In this tutorial we work with dummy data in order to show results

In [8]:
with open(join('..','tests','data','demo','ARS.pkl'), 'rb') as handle:
    df_ARS = pickle.load(handle)
df_ARS

,entity,period,categorie,"S.02.01.01.01,R0010,C0020","S.02.01.01.01,R0020,C0020","S.02.01.01.01,R0030,C0010","S.02.01.01.01,R0030,C0020","S.02.01.01.01,R0040,C0010","S.02.01.01.01,R0040,C0020","S.02.01.01.01,R0050,C0010",...,"S.29.03.01.06,R0330,C0100","S.29.03.01.06,R0330,C0110","S.29.03.01.06,R0340,C0100","S.29.03.01.06,R0340,C0110","S.29.03.01.06,R0350,C0100","S.29.03.01.06,R0350,C0110","S.29.03.01.07,R0360,C0120","S.29.03.01.07,R0360,C0130","S.29.03.01.07,R0370,C0120","S.29.03.01.07,R0370,C0130"
0,0LFF1WMNTWG5PTIYYI38,2019-12-31,Schade,4.282743e+08,4.593296e+08,970830023.1,9.485834e+07,8.708247e+08,5.494963e+08,1.168024e+08,...,5.852015e+08,7.837748e+08,7.128474e+08,65836985.03,9.328439e+08,5.850669e+08,56406511.08,8.012159e+08,8.769882e+08,6.403964e+08
0,0LFF1WMNTWG5PTIYYI38,2018-12-31,Schade,4.432743e+08,4.743296e+08,985830023.1,1.098583e+08,8.858247e+08,5.644963e+08,1.318024e+08,...,6.002015e+08,7.987748e+08,7.278474e+08,80836985.03,9.478439e+08,6.000669e+08,71406511.08,8.162159e+08,8.919882e+08,6.553964e+08


#### S2_betweenperiods_QRS

In [9]:
# df_datapoints = pd.read_csv(join(DATAPOINTS_PATH, 'QRS.csv'), sep=";").fillna("")  # load file to dataframe
# dft = pd.DataFrame()
# for instance in instances_QRS:
#     df_closed_axis = pd.DataFrame()
#     tables_closed_axis = []  # for listing all input tables with closed axis
#     # get tables
#     tables_complete_set = df_datapoints.tabelcode.sort_values().unique().tolist()  # list of all QRS tables
#     tables = [table for table in tables_complete_set 
#         if isfile(join(INSTANCES_DATA_PATH,instance,table + '.pickle'))]  # QRS tables found in the specified INSTANCES_DATA_PATH
#     for table in [table for table in tables if table not in ['S.14.01.01.04','S.30.03.01.01']]:  #tables:
#         if isfile(join(INSTANCES_DATA_PATH,instance, table + '.pickle')):
#             df = pd.read_pickle(join(INSTANCES_DATA_PATH,instance, table + '.pickle'))  # read dataframe
#         else:
#             continue
    
#         if df.index.nlevels > 2:  # if more than 2 indexes (entity, period), then the table has an open axis
#             continue
#         else:  # closed axis
#             tables_closed_axis.append(table)  # add to relevant list
        
#             # Add table to dataframe with all data from closed axis tables
#             if len(df_closed_axis) == 0:  # no data yet --> copy dataframe
#                 df_closed_axis = df.copy()
#             else:  # join to existing dataframe
#                 df_closed_axis = df_closed_axis.join(df)
#     if len(dft) == 0:  # no data yet 
#         dft = df_closed_axis
#     else:  # join to existing dataframe
#         dft=dft.append(df_closed_axis)
# dft=dft.reset_index()
# dft['categorie']='Schade'
# numerical_columns = ['entity','period','categorie'] + [dft.columns[c] for c in range(len(dft.columns))
#                         if ((dft.dtypes[c] == 'float64') or (dft.dtypes[c] == 'int64'))]
# df_QRS = dft[numerical_columns]
# df_QRS['period']=df_QRS['period'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d')) #convert to datetime
# df_QRS.fillna(0,inplace=True)

In this tutorial we work with dummy data in order to show results

In [10]:
with open(join('..','tests','data','demo','QRS.pkl'), 'rb') as handle:
    df_QRS = pickle.load(handle)
df_QRS

,entity,period,categorie,"S.02.01.02.01,R0030,C0010","S.02.01.02.01,R0040,C0010","S.02.01.02.01,R0050,C0010","S.02.01.02.01,R0060,C0010","S.02.01.02.01,R0070,C0010","S.02.01.02.01,R0080,C0010","S.02.01.02.01,R0090,C0010",...,"S.28.02.01.06,R0520,C0140","S.28.02.01.06,R0520,C0150","S.28.02.01.06,R0530,C0140","S.28.02.01.06,R0530,C0150","S.28.02.01.06,R0540,C0140","S.28.02.01.06,R0540,C0150","S.28.02.01.06,R0550,C0140","S.28.02.01.06,R0550,C0150","S.28.02.01.06,R0560,C0140","S.28.02.01.06,R0560,C0150"
0,0LFF1WMNTWG5PTIYYI38,2019-12-31,Schade,5.132284e+08,8.163135e+08,7.385397e+08,5.002937e+08,8.875499e+07,3.685431e+08,7.201291e+08,...,5.854466e+08,4.441183e+08,5.668300e+08,7.504875e+08,9.070768e+08,2.163901e+08,563403902.1,9.515222e+08,5.487391e+08,5.306051e+08
0,0LFF1WMNTWG5PTIYYI38,2018-12-31,Schade,5.282284e+08,8.313135e+08,7.535397e+08,5.152937e+08,1.037550e+08,3.835431e+08,7.351291e+08,...,6.004466e+08,4.591183e+08,5.818300e+08,7.654875e+08,9.220768e+08,2.313901e+08,578403902.1,9.665222e+08,5.637391e+08,5.456051e+08


### Evaluate rules

#### Evaluate S2_betweenperiods_ARS

In [11]:
miner = data_patterns.PatternMiner(df_patterns=dfr_ARS)
miner.df_data = df_ARS
miner.metapatterns = {'cluster':'categorie'}
miner.convert_to_time(['entity', 'categorie'], 'period')
miner.df_data = miner.df_data.reset_index()

results = miner.analyze()
results

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13771/13771 [01:36<00:00, 143.28it/s]


,result_type,pattern_id,cluster,support,exceptions,confidence,pattern_def,P values,Q values
index,,,,,,,,,
0,True,"S.02.01.01.01,R0030,C0010 consistently reported",Schade,1,0,1.0,"(({""S.02.01.01.01,R0030,C0010 (t)""} != 0) & ({...","[985830023.1, 970830023.1, 985830023.1, 970830...",[]
0,True,"S.02.01.01.01,R0050,C0010 consistently reported",Schade,1,0,1.0,"(({""S.02.01.01.01,R0050,C0010 (t)""} != 0) & ({...","[131802409.52, 116802409.52, 131802409.52, 116...",[]
0,True,"S.02.01.01.01,R0060,C0010 consistently reported",Schade,1,0,1.0,"(({""S.02.01.01.01,R0060,C0010 (t)""} != 0) & ({...","[92093842.97, 77093842.97, 92093842.97, 770938...",[]
0,True,"S.02.01.01.01,R0070,C0010 consistently reported",Schade,1,0,1.0,"(({""S.02.01.01.01,R0070,C0010 (t)""} != 0) & ({...","[354431184.66, 339431184.66, 354431184.66, 339...",[]
0,True,"S.02.01.01.01,R0080,C0010 consistently reported",Schade,1,0,1.0,"(({""S.02.01.01.01,R0080,C0010 (t)""} != 0) & ({...","[831264610.4, 816264610.4, 831264610.4, 816264...",[]
...,...,...,...,...,...,...,...,...,...
0,False,"Delta-S.27.01.01.22,R4360,C1500 < 10%",Schade,0,1,0.0,"IF ({""S.27.01.01.22,R4360,C1500 (t)""} != 0) & ...","[15000000.947, 0.947]","[15000000.947, 0.947, 0.947]"
0,False,"Delta-S.27.01.01.22,R4370,C1500 < 10%",Schade,0,1,0.0,"IF ({""S.27.01.01.22,R4370,C1500 (t)""} != 0) & ...","[15000000.7561, 0.7561]","[15000000.7561, 0.7561, 0.7561]"
0,False,"Delta-S.27.01.01.22,R4380,C1500 < 10%",Schade,0,1,0.0,"IF ({""S.27.01.01.22,R4380,C1500 (t)""} != 0) & ...","[15000000.3359, 0.3359]","[15000000.3359, 0.3359, 0.3359]"


#### S2_betweenperiods_QRS

In [12]:
miner = data_patterns.PatternMiner(df_patterns=dfr_QRS)
miner.df_data = df_QRS
miner.metapatterns = {'cluster':'categorie'}
miner.convert_to_time(['entity', 'categorie'], 'period', set_year=False)
miner.df_data = miner.df_data.reset_index()

results = miner.analyze()
results

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2856/2856 [00:12<00:00, 223.26it/s]


,result_type,pattern_id,cluster,support,exceptions,confidence,pattern_def,P values,Q values
index,,,,,,,,,
0,True,"S.02.01.02.01,R0030,C0010 consistently reported",Schade,1,0,1.0,"(({""S.02.01.02.01,R0030,C0010 (t)""} != 0) & ({...","[528228409.32, 513228409.32, 528228409.32, 513...",[]
0,True,"S.02.01.02.01,R0040,C0010 consistently reported",Schade,1,0,1.0,"(({""S.02.01.02.01,R0040,C0010 (t)""} != 0) & ({...","[831313543.58, 816313543.58, 831313543.58, 816...",[]
0,True,"S.02.01.02.01,R0050,C0010 consistently reported",Schade,1,0,1.0,"(({""S.02.01.02.01,R0050,C0010 (t)""} != 0) & ({...","[753539746.57, 738539746.57, 753539746.57, 738...",[]
0,True,"S.02.01.02.01,R0060,C0010 consistently reported",Schade,1,0,1.0,"(({""S.02.01.02.01,R0060,C0010 (t)""} != 0) & ({...","[515293694.25, 500293694.25, 515293694.25, 500...",[]
0,True,"S.02.01.02.01,R0070,C0010 consistently reported",Schade,1,0,1.0,"(({""S.02.01.02.01,R0070,C0010 (t)""} != 0) & ({...","[103754986.46, 88754986.46, 103754986.46, 8875...",[]
...,...,...,...,...,...,...,...,...,...
0,True,"Delta-S.28.01.01.05,R0350,C0070 < 10%",Schade,1,0,1.0,"IF ({""S.28.01.01.05,R0350,C0070 (t)""} != 0) & ...","[431520867.15, 416520867.15]","[431520867.15, 416520867.15, 416520867.15]"
0,True,"S.28.01.01.05,R0350,C0070 consistently reported",Schade,1,0,1.0,"(({""S.28.01.01.05,R0350,C0070 (t)""} != 0) & ({...","[431520867.15, 416520867.15, 431520867.15, 416...",[]
0,True,"S.28.01.01.05,R0400,C0070 consistently reported",Schade,1,0,1.0,"(({""S.28.01.01.05,R0400,C0070 (t)""} != 0) & ({...","[314152046.19, 299152046.19, 314152046.19, 299...",[]
